In [1]:
import re

# 1. Regex basics

In [2]:
out = re.match(r"hello", "hello world")

print(
    out.start(),
    out.end(),
    bool(out)
)

0 5 True


In [3]:
regex = r"hello"

for text in ["hello world", "bye world"]:
    out = re.match(regex, text)
    
    print(text, "\tFound:", bool(out))

hello world 	Found: True
bye world 	Found: False


## 1.1. `match` vs `search`

In [4]:
text = """hello world
bye world"""

print(bool(re.match(r"hello", text)))
print(bool(re.match(r"world", text)))
print(bool(re.match(r"bye", text)))

print(bool(re.search(r"hello", text)))
print(bool(re.search(r"world", text)))
print(bool(re.search(r"bye", text)))

True
False
False
True
True
True


## 1.2. `re.compile`

In [5]:
%timeit re.match(r"hello", "hello world")

820 ns ± 14.2 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [6]:
regex = re.compile(r"hello")

In [7]:
%timeit regex.match("hello world")

338 ns ± 5.05 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


## 1.3. Groups

In [8]:
filenames = [
    "Informe mensual Indexa Capital - AABBCCDD - 2020-01.pdf",
    "Informe mensual Indexa Capital - XXYYZZWW - 2020-01.pdf",
    "Informe mensual Indexa Capital - AABBCCDD - 2021-03.pdf",
    "Informe mensual Indexa Capital - XXYYZZWW - 2020-12.pdf",
]

### 1.3.1. Unnamed groups

In [9]:
regex = re.compile(r"Informe mensual Indexa Capital - (\w{8}) - (\d{4})-(\d{2}).pdf")

In [10]:
out = regex.match(filenames[0])
out.groups()

('AABBCCDD', '2020', '01')

### 1.3.2. Named groups

In [11]:
regex = re.compile(r"Informe mensual Indexa Capital - (?P<account>\w{8}) - (?P<year>\d{4})-(?P<month>\d{2}).pdf")

In [12]:
out = regex.match(filenames[1])
out.groupdict()

{'account': 'XXYYZZWW', 'year': '2020', 'month': '01'}

### 1.3.3. Using named groups

In [13]:
out_pattern = "{year}_{month} Indexa capital {account}.pdf"

In [14]:
for filename in filenames:
    groups = regex.match(filename).groupdict()
    new_name = out_pattern.format(**groups)
    
    print(filename, "\n-", new_name, "\n")

Informe mensual Indexa Capital - AABBCCDD - 2020-01.pdf 
- 2020_01 Indexa capital AABBCCDD.pdf 

Informe mensual Indexa Capital - XXYYZZWW - 2020-01.pdf 
- 2020_01 Indexa capital XXYYZZWW.pdf 

Informe mensual Indexa Capital - AABBCCDD - 2021-03.pdf 
- 2021_03 Indexa capital AABBCCDD.pdf 

Informe mensual Indexa Capital - XXYYZZWW - 2020-12.pdf 
- 2020_12 Indexa capital XXYYZZWW.pdf 



# 2. Pandas

In [15]:
import pandas as pd

In [16]:
queries = [
    "select *  from  users",
    "SELECT count(id) AS cnt FROM public.users",
    "SELECT * FROM orders",
    "SELECT city, count(1) FROM orders GROUP BY 1",
]
df = pd.DataFrame(queries, columns=["query"])
df

,query
0,select * from users
1,SELECT count(id) AS cnt FROM public.users
2,SELECT * FROM orders
3,"SELECT city, count(1) FROM orders GROUP BY 1"


## 2.1. series.str.count

In [17]:
df["query"].str.count(r"\s+FROM\s+(\w+\.)?users\b")

0    0
1    1
2    0
3    0
Name: query, dtype: int64

## 2.2. series.str.replace

In [18]:
df["query"].str.replace(r"\s+AS\s+\w+\b", "", regex=True)

0                           select *  from  users
1              SELECT count(id) FROM public.users
2                            SELECT * FROM orders
3    SELECT city, count(1) FROM orders GROUP BY 1
Name: query, dtype: object

## 2.3. series.str.match

In [19]:
df["query"].str.match(r"(?i)(SELECT)\s+\*\s+(?i)(FROM)")

0     True
1    False
2     True
3    False
Name: query, dtype: bool

## 2.4. series.str.contains

In [20]:
df["query"].str.contains(r"\s+AS\s+\w+")

0    False
1     True
2    False
3    False
Name: query, dtype: bool

## 2.5. series.str.findall

In [21]:
df["query"].str.findall("\w{5,8}")

0                   [select, users]
1    [SELECT, count, public, users]
2                  [SELECT, orders]
3    [SELECT, count, orders, GROUP]
Name: query, dtype: object

## 2.6. series.str.split / series.str.rsplit

In [22]:
df["query"].str.split("\s+(?i)from\s+")

0                             [select *, users]
1       [SELECT count(id) AS cnt, public.users]
2                            [SELECT *, orders]
3    [SELECT city, count(1), orders GROUP BY 1]
Name: query, dtype: object

## 2.7. series.str.extract

In [23]:
regex = r"(?i)SELECT\s+(?:.+)\s+(?i)FROM\s+(?:(?P<schema>\w+)\.)?(?P<table>\w+)\b"
df["query"].str.extract(regex)

,schema,table
0,NaN,users
1,public,users
2,NaN,orders
3,NaN,orders
